In [71]:
!pip install prettytable
from prettytable import PrettyTable
import math

In [88]:
#Номинальные ряды
E24 = (10, 11, 12, 13, 15, 16, 18, 20, 22, 24, 27, 30, 33, 36, 39, 43, 47, 51, 56, 62, 68, 75, 82, 91)
E12 = (10, 12, 15, 18, 22, 27, 33, 39, 47, 56, 68, 75, 82, 91)
E6 = (10, 15, 22, 33, 47, 68)


def choose_nom_value(E, x):
    n = 0
    while(x < E[0]):
        x = x*10
        n  = n-1
    while(x > E[-1]):
        x = x/10
        n = n+1
        
    for val in E:
        if(x <= val):
            return val*10**n
    return -1

20000


In [103]:
#Заданные величины - Ярослав
# максимальное напряжение
U_m = 4.4
# сопротивление нагрузки
R_n = 0.55 * 10**3
#внутреннее сопротивление источника сигнала
R_g = 1.6 * 10**3
# макс отклонение температуры от 20 градусов
dT = 60
#коэф усилиния транзистора по току
h_fe = (40, 120)
#температурный коэффициент 
xi = 2*10**-3



In [120]:
#Заданные величины - Валера
# максимальное напряжение
U_m = 4.5
# сопротивление нагрузки
R_n = 0.6 * 10**3
#внутреннее сопротивление источника сигнала
R_g = 1.6 * 10**3
# макс отклонение температуры от 20 градусов
dT = 60
#коэф усилиния транзистора по току
h_fe = (30, 150)
#температурный коэффициент 
xi = 2*10**-3
f_n = 350 

C_ep = 0
#емкость коллекторного перехода
C_kp = 25*10**-12
#емкость нагрузки
C_n = 7 * 10**-9

#граничная частота
f_gr = 15 *10**6

# Пункт 4

In [75]:
#Рабочая точка
#отклонение напряжения нагрузки в связи с нелинейностью
dU_n = 1
#отклонение напряжения из-за температуры
dU_t = 1
#напряжение коллектор эммитер
U_ke = 7
#ток коллектора
I_k = 20 * 10**-3
#напряжение питания
E_p = 18
U_be = -0.620
table = PrettyTable()
table.field_names = ["|U_кэА| Напряжение коллектор-эммитер [В]", "I_kТок коллектора [мА]", "|Eпит| Напряжение питания [B] "]
table.add_row([U_ke, I_k*10**3, E_p])
print(table)


+------------------------------------------+------------------------+--------------------------------+
| |U_кэА| Напряжение коллектор-эммитер [В] | I_kТок коллектора [мА] | |Eпит| Напряжение питания [B]  |
+------------------------------------------+------------------------+--------------------------------+
|                    7                     |          20.0          |               18               |
+------------------------------------------+------------------------+--------------------------------+


# Пункт 5

$$R_{=}= \frac{ (E_{пит} - U_{кэА} )}{I_{кэА}}$$

$$ R_{=} =R_{э}$$

$$ R_{\~} = \frac{R_{э}*R_{H}}{R_{э}+R_{H}}$$

$$ U_{кэВ} = |I_{k}| *R_{\~} + |U_{кэА}|$$

In [91]:
#Эквивалентное сопротивление каскада
R_dc = (E_p-abs(U_ke))/I_k

#выбор из номинального ряда E24
R_e = choose_nom_value(E24, R_dc)

        
#R тильда       
R_ac = R_e*R_n/(R_e+R_n)

#расчет доп точки для построения нагрузочной характеристики
U_keB = abs(I_k)*R_ac + U_ke


table = PrettyTable()
table.field_names = ["R_= Посчитанное значение[Ом]", " R_э Значение из ряда Е24  [Oм]", "R_~ [Ом] ", "U_кэВ [B]"]
table.add_row([R_dc, R_e, R_ac, U_keB])
print(table)

+------------------------------+---------------------------------+-------------------+--------------------+
| R_= Посчитанное значение[Ом] |  R_э Значение из ряда Е24  [Oм] |     R_~ [Ом]      |     U_кэВ [B]      |
+------------------------------+---------------------------------+-------------------+--------------------+
|            550.0             |               560               | 289.6551724137931 | 12.793103448275861 |
+------------------------------+---------------------------------+-------------------+--------------------+


# Пункт 6


In [92]:
table = PrettyTable()

#допустимое температурное изменение тока коллектора.
dI_k = dU_t/R_e

table.field_names = ["Переменная", "Ед измерения", "Значение"]
table.add_row(["dI_k Доп изменение тока коллектора", "мA",dI_k *10**3], divider=True)

#Коэфициент бета
beta = (h_fe[1]+h_fe[0])/2
table.add_row(["Beta коэффициент усиления по току.", "-", beta], divider=True)

#максимальное отклонение бета
dbeta = h_fe[1]-beta
table.add_row(["dBeta макс отклонение Beta.", "-", dbeta], divider=True)

#Ток базы
I_b = I_k/beta
table.add_row(["ток базы.", "мкА", I_b*10**6], divider=True)


#сопротивление базы
R_b = (beta*(dI_k*R_e-xi*dT))/(I_b*dbeta-dI_k) - R_e

table.add_row(["Сопротивление базы.", "Ом", R_b], divider=True)



I_e = I_b + I_k
table.add_row(["Ток эммитера", "мA", I_e*10**3], divider=True)

R1 = E_p/ ((I_e*R_e + U_be)/R_b + I_b)
table.add_row(["Сопротивление R1.", "Ом", R1], divider=True)

R2 = R_b*R1/(R1-R_b)
table.add_row(["Сопротивление R2.", "Ом", R2], divider=True)

R1 = choose_nom_value(E24, R1)
R2 = choose_nom_value(E24, R2)


table.add_row(["Сопротивление R1 из номинального ряда Е24.", "Ом", R1], divider=True)
table.add_row(["Сопротивление R2 из номинального ряда Е24.", "Ом", R2], divider=True)
print(table)

+--------------------------------------------+--------------+--------------------+
|                 Переменная                 | Ед измерения |      Значение      |
+--------------------------------------------+--------------+--------------------+
|     dI_k Доп изменение тока коллектора     |      мA      | 1.7857142857142856 |
+--------------------------------------------+--------------+--------------------+
|     Beta коэффициент усиления по току.     |      -       |        90.0        |
+--------------------------------------------+--------------+--------------------+
|        dBeta макс отклонение Beta.         |      -       |        60.0        |
+--------------------------------------------+--------------+--------------------+
|                 ток базы.                  |     мкА      | 222.22222222222223 |
+--------------------------------------------+--------------+--------------------+
|            Сопротивление базы.             |      Ом      | 6298.556701030928  |
+---

# Пункт 7

In [78]:
table = PrettyTable()
table.field_names = ["Переменная", " Ед измерения", "Значение до п7", "Значение в п7", "Отклонение"]
#непряжение база-эммитер
U_be_p7 = -0.7
#напряжение базы
E_b_p7 = E_p * R2/(R1+R2)
print(E_b_p7)
#ток базы
I_b_p7 = (E_b_p7-U_be_p7)/(R_b+(1+beta)*R_e)
#ток коллектора
I_k_p7 = beta*I_b_p7
#ток эммитера
I_e_p7 = I_k_p7 + I_b_p7
print(I_e_p7)
#напряжение коллектор-эммитер
U_ke_p7 = E_p - I_e_p7 * R_e


table.add_row(["Напряжение база-эммитер", "В", U_be, U_be_p7, abs(U_be - U_be_p7)], divider=True)
table.add_row(["Ток базы", "мкА", I_b* 10**6, I_b_p7* 10**6, abs(I_b - I_b_p7)* 10**6], divider=True)
table.add_row(["Напряжение коллектор-эммитер", "В", U_ke, U_ke_p7, abs(U_ke - U_ke_p7)], divider=True)
table.add_row(["Ток коллектора", "мА", I_k* 10**3, I_k_p7* 10**3, abs(I_k - I_k_p7)* 10**3], divider=True)

print(table)


12.0
0.02018388283928211
+------------------------------+---------------+--------------------+--------------------+---------------------+
|          Переменная          |  Ед измерения |   Значение до п7   |   Значение в п7    |      Отклонение     |
+------------------------------+---------------+--------------------+--------------------+---------------------+
|   Напряжение база-эммитер    |       В       |       -0.62        |        -0.7        | 0.07999999999999996 |
+------------------------------+---------------+--------------------+--------------------+---------------------+
|           Ток базы           |      мкА      | 222.22222222222223 | 221.80091032178143 | 0.42131190044079264 |
+------------------------------+---------------+--------------------+--------------------+---------------------+
| Напряжение коллектор-эммитер |       В       |         7          | 6.697025610002019  |  0.3029743899979813 |
+------------------------------+---------------+--------------------+--

# Пункт 8

In [79]:
table = PrettyTable()
table.field_names = ["Переменная", "Ед измерения", "Значение"]

#уход рабочей точки из-за влияния температуры
dI_T_p8 = abs(xi*dT)/(R_b+R_e*(1+beta))
table.add_row(["dI_T уход рабочей точки из-за влияния температуры", "мA",dI_T_p8 *10**3], divider=True)

#коэффициент перераспределения тока
gamma = (R_e )/(R_e+R_b)
table.add_row(["gamma коэффициент перераспределения тока", "-",gamma], divider=True)

# уход рабочей точки из технологического разброса параметра Beta
dI_BETA_p8 = (dbeta * I_k_p7)/(beta*(1+gamma*beta))
table.add_row(["уход рабочей точки из технологического разброса параметра Beta", "мА",dI_BETA_p8 * 10**3], divider=True)


#уход тока коллектора
dI_k_p8 = dI_T_p8 + dI_BETA_p8
table.add_row(["уход тока коллектора", "мА",dI_k_p8 * 10**3], divider=True)

#уход напряжения из-за колебаний температуры
dU_T_p8 = dI_k_p8*R_dc
table.add_row(["уход напряжения из-за колебаний температуры", "В",dU_T_p8], divider=True)


#Переменное напряжение
dU_nmax = R_ac*I_k - dU_T_p8
table.add_row(["Переменное напряжение", "В", dU_nmax], divider=True)

print(table)

+----------------------------------------------------------------+--------------+-----------------------+
|                           Переменная                           | Ед измерения |        Значение       |
+----------------------------------------------------------------+--------------+-----------------------+
|       dI_T уход рабочей точки из-за влияния температуры        |      мA      | 0.0020957566329617146 |
+----------------------------------------------------------------+--------------+-----------------------+
|            gamma коэффициент перераспределения тока            |      -       |  0.08164983164983165  |
+----------------------------------------------------------------+--------------+-----------------------+
| уход рабочей точки из технологического разброса параметра Beta |      мА      |   1.5940682484106252  |
+----------------------------------------------------------------+--------------+-----------------------+
|                      уход тока коллектора   

# Пункт 9

In [80]:
table = PrettyTable()
table.field_names = ["Переменная", "Ед измерения", "Значение"]

U1 = 602.916*10**-3
U2 = 565.1*10**-3
I1= 200*10**-6
I2 = 50*10**-6

dU = abs(U1 - U2)
table.add_row(["Изменение напряжения", "мВ",dU * 10**3], divider=True)
dI = abs(I1 - I2)
table.add_row(["Изменение тока", "мкА",dI * 10**6], divider=True)

h_11e = dU/dI
h_21e = beta
table.add_row(["Параметр h_11e", "Ом", dh_11e ], divider=True)


R_in = (R_b * (h_11e + ((1+h_21e) * R_e * R_n) / (R_e + R_n) ) )/( R_b + (h_11e + ((1+h_21e) * R_e * R_n) / (R_e + R_n) ))
table.add_row(["Сопротивление входа", "Ом", R_in ], divider=True)

R_out = (R_e * (h_11e + (R_g*R_b)/(R_g+R_b))/(1+h_21e))/(R_e + (h_11e + (R_g*R_b)/(R_g+R_b))/(1+h_21e))
table.add_row(["Сопротивление выхода", "Ом", R_out ], divider=True)


K_ruxx = ((1+h_21e)*(R_e*R_n)/(R_e + R_n))/(h_11e+ (1+h_21e)*(R_e*R_n)/(R_e + R_n))
table.add_row(["K_ruxx", "-", K_ruxx ], divider=True)

epsilon_in = 1.03*(R_in)/(R_in + R_g)
table.add_row(["Epsilon входа", "-", epsilon_in ], divider=True)

epsilon_out = (R_n)/(R_n + R_out)
table.add_row(["Epsilon выхода", "-", epsilon_out ], divider=True)

#K_e0 = K_ruxx * epsilon_in * epsilon_out 
#костыль 
K_e0 = 0.7708328538666575
table.add_row(["K_e0", "-", K_e0 ], divider=True)

print(table)

+----------------------+--------------+--------------------+
|      Переменная      | Ед измерения |      Значение      |
+----------------------+--------------+--------------------+
| Изменение напряжения |      мВ      | 37.816000000000074 |
+----------------------+--------------+--------------------+
|    Изменение тока    |     мкА      |       150.0        |
+----------------------+--------------+--------------------+
|    Параметр h_11e    |      Ом      | 252.10666666666717 |
+----------------------+--------------+--------------------+
| Сопротивление входа  |      Ом      | 5093.066589275127  |
+----------------------+--------------+--------------------+
| Сопротивление выхода |      Ом      | 16.30235998948666  |
+----------------------+--------------+--------------------+
|        K_ruxx        |      -       | 0.9905261264267257 |
+----------------------+--------------+--------------------+
|    Epsilon входа     |      -       | 0.7837750479517519 |
+----------------------+

# Пункт 10


In [101]:
table = PrettyTable()
table.field_names = ["Переменная", "Ед измерения", "Значение"]

tau_n  = 1/(f_n * 2 * math.pi)
tau_n1 = 2*tau_n
tau_n2 = tau_n1
table.add_row(["tau_n", "мс",tau_n*10**3], divider=True)
table.add_row(["taun_n1 = tau_n2", "мс",tau_n1*10**3], divider=True)

C_p1 = (tau_n1)/(R_in + R_g)
table.add_row(["C_p1", "нФ",C_p1*10**9 ], divider=True)
C_p1 = choose_nom_value(E12, C_p1)
table.add_row(["C_p1 из номинального ряда Е12", "нФ",C_p1*10**9 ], divider=True)


C_p2 = (tau_n2)/(R_out + R_n)
table.add_row(["C_p2", "мкФ",C_p2*10**6 ], divider=True)
#выбор из ном ряда Е6
C_p2 = choose_nom_value(E6, C_p2)
table.add_row(["C_p2 из номинального ряда Е6", "мкФ",C_p2*10**6 ], divider=True)

print(table)

+-------------------------------+--------------+---------------------+
|           Переменная          | Ед измерения |       Значение      |
+-------------------------------+--------------+---------------------+
|             tau_n             |      мс      | 0.45472840883398674 |
+-------------------------------+--------------+---------------------+
|        taun_n1 = tau_n2       |      мс      |  0.9094568176679735 |
+-------------------------------+--------------+---------------------+
|              C_p1             |      нФ      |  135.88043769432596 |
+-------------------------------+--------------+---------------------+
| C_p1 из номинального ряда Е12 |      нФ      |        150.0        |
+-------------------------------+--------------+---------------------+
|              C_p2             |     мкФ      |  1.475666615463695  |
+-------------------------------+--------------+---------------------+
|  C_p2 из номинального ряда Е6 |     мкФ      |         1.5         |
+-----

# Пункт 11

In [126]:
table = PrettyTable()
table.field_names = ["Переменная", "Ед измерения", "Значение"]

tau_v1 = ( (C_ep + C_kp*K_e0)*R_in*R_g )/(R_in+R_g) 
table.add_row(["tau_v1", "нC", tau_v1*10**9 ], divider=True)

C_nekv = C_n +C_kp*beta
table.add_row(["C_nekv", "нФ",C_nekv*10**9 ], divider=True)

tau_v2 = C_nekv * (R_n * R_out)/(R_n + R_out)
table.add_row(["tau_v2", "нC",tau_out*10**9 ], divider=True)


tau_t = beta/(2*math.pi * f_gr * (1 + beta*(R_e / (R_e+R_b))) )
table.add_row(["tau_t", "нC", tau_t*10**9 ], divider=True)

tau_v = (tau_v1**2 + tau_v2**2 + tau_t**2)**0.5
table.add_row(["tau_v", "нC", tau_v*10**9 ], divider=True)


f_v_p11 = 1/(2*math.pi*tau_v)
table.add_row(["f_в", "кГц", f_v_p11*10**-3], divider=True)
print(table)

+------------+--------------+--------------------+
| Переменная | Ед измерения |      Значение      |
+------------+--------------+--------------------+
|   tau_v1   |      нC      | 23.462507068043696 |
+------------+--------------+--------------------+
|   C_nekv   |      нФ      |        9.25        |
+------------+--------------+--------------------+
|   tau_v2   |      нC      | 146.80796929480266 |
+------------+--------------+--------------------+
|   tau_t    |      нC      | 114.38358886459268 |
+------------+--------------+--------------------+
|   tau_v    |      нC      |  187.581114422337  |
+------------+--------------+--------------------+
|    f_в     |     кГц      | 848.4593109600552  |
+------------+--------------+--------------------+
